In [ ]:
pip install requests bs4

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_caic_weather_data(url):
    # Fetch the webpage content
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    all_data = []
    current_zone = None

    # Find all table elements
    tables = soup.find_all('table', class_='sortable')

    for table in tables:
        # Extract the zone name from the preceding h4 element
        zone_header = table.find_previous('h4')
        if zone_header:
            current_zone = zone_header.text.strip()

        # Extract column headers
        headers = [th.text.strip() for th in table.find_all('th')]

        # Extract rows
        rows = table.find_all('tr')[1:]  # Skip the header row

        # Process each row
        for row in rows:
            cols = row.find_all('td')
            if len(cols) >= len(headers):
                data = {headers[i]: cols[i].text.strip() for i in range(len(headers))}
                data['Zone'] = current_zone
                all_data.append(data)

    # Create a DataFrame
    df = pd.DataFrame(all_data)

    return df

# URL of the CAIC weather stations page
url = "https://classic.avalanche.state.co.us/caic/obs_stns/zones.php"

# Scrape the data
try:
    weather_data = scrape_caic_weather_data(url)
    
    # Save to CSV
    weather_data.to_csv("caic_weather_data.csv", index=False)
    print("Data has been scraped and saved to 'caic_weather_data.csv'")
    
    # Display first few rows
    print("\nFirst few rows of the scraped data:")
    print(weather_data.head())
    
    # Display summary information
    print("\nDataset summary:")
    print(weather_data.info())

except Exception as e:
    print(f"An error occurred while scraping the data: {str(e)}")